<a href="https://colab.research.google.com/github/Safeenaa07/samplerepo/blob/main/Activity3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ---------------------------------------------------------
# SENTIMENT ANALYSIS USING BIDIRECTIONAL LSTM (BiLSTM)
# TRAINED ON REAL IMDB DATASET (50,000 reviews)
# ---------------------------------------------------------

import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

# ---------------------------------------------------------
# 1. LOAD IMDB DATASET
# ---------------------------------------------------------
vocab_size = 10000  # top 10k words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

max_len = 200  # long reviews → need longer max length

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# ---------------------------------------------------------
# 2. BUILD BiLSTM MODEL
# ---------------------------------------------------------
model = Sequential([
    Embedding(vocab_size, 128, input_length=max_len),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Force-build model
model.build(input_shape=(None, max_len))
model.summary()

# ---------------------------------------------------------
# 3. TRAIN MODEL
# ---------------------------------------------------------
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.fit(
    X_train, y_train,
    epochs=3,        # IMDB is large, even 3 epochs works well
    batch_size=128,
    validation_data=(X_test, y_test)
)

# ---------------------------------------------------------
# 4. FUNCTION TO PREDICT SENTIMENT
# ---------------------------------------------------------

# Load IMDB word index
word_index = imdb.get_word_index()
index_to_word = {v+3: k for k, v in word_index.items()}
index_to_word[0], index_to_word[1], index_to_word[2], index_to_word[3] = "<PAD>", "<START>", "<UNK>", "<UNUSED>"

def encode_review(text):
    words = text.lower().split()
    encoded = [1]  # start token
    for w in words:
        if w in word_index:
            encoded.append(word_index[w] + 3)
        else:
            encoded.append(2)   # unknown
    return pad_sequences([encoded], maxlen=max_len)

def predict_sentiment(text):
    encoded = encode_review(text)
    prediction = float(model.predict(encoded)[0])

    if prediction > 0.5:
        return f"Positive 😊 (score={prediction:.4f})"
    else:
        return f"Negative 😠 (score={prediction:.4f})"

# ---------------------------------------------------------
# 5. TEST PREDICTIONS
# ---------------------------------------------------------

print(predict_sentiment("This was the worst film ever"))
print(predict_sentiment("I absolutely loved this movie!"))
print(predict_sentiment("Not great, but not terrible either"))


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,387,137 (5.29 MB)

 Trainable params: 1,387,137 (5.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 189s 940ms/step - accuracy: 0.7002 - loss: 0.5332 - val_accuracy: 0.8688 - val_loss: 0.3111
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 184s 938ms/step - accuracy: 0.9139 - loss: 0.2221 - val_accuracy: 0.8679 - val_loss: 0.3158
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 184s 939ms/step - accuracy: 0.9419 - loss: 0.1595 - val_accuracy: 0.8678 - val_loss: 0.3505
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
Negative 😠 (score=0.0510)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Positive 😊 (score=0.7755)


/tmp/ipython-input-3533018758.py:76: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediction = float(model.predict(encoded)[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Negative 😠 (score=0.0835)
